In [ ]:
import pandas as pd
import datetime
import time
import csv     
import os.path
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import StackingClassifier

#Load dataset
dataset1 = pd.read_csv("labelled1.csv")

features = dataset1.columns.tolist()
features.remove('timestamp'), features.remove('class')
print(features)

#Import Random Forest Model
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

#GB
clf1 =  GradientBoostingClassifier()
#DTC
clf2 =  DecisionTreeClassifier()
#Naive Bayes
clf3 =  GaussianNB()
clf = OneVsRestClassifier(StackingClassifier(estimators=[('gb',clf1), ('dtc', clf2),('gnb', clf3)], cv=10))
#Train the model using the training sets
clf.fit(dataset1[features], dataset1['class'])

def periodic_check(interval):
    while True:
        dataset2 = pd.read_csv("data.csv")
        dataset2_size = dataset2.shape[0]

        if os.path.isfile("data_copy.csv"):
            dataset3 = pd.read_csv("data_copy.csv")
            dataset3_size = dataset3.shape[0]
        else:
            dataset3_size = 0
            with open("data_copy.csv", "w"):
                pass
        
        if dataset2_size != dataset3_size:
            with open('data.csv', 'r') as csvfile1:
                with open ("data_copy.csv", "r") as csvfile2:
                    reader1 = csv.reader(csvfile1)
                    reader2 = csv.reader(csvfile2)
                    rows1 = [row for row in reader1]
                    rows2 = [row for row in reader2]
                    new_rows = []                 
                    for row in rows1:
                        if row not in rows2:
                            new_rows.append(row)
                            
        if(new_rows[0][0] == 'timestamp'):
            del new_rows[0]

        df = pd.DataFrame(new_rows,columns=['timestamp', 'tank_liquidlevel', 'flowlevel', 'bottle_liquidlevel', 'motor_status'])
        if df.shape[0] != 0:
            pred = clf.predict(df[features])
            df['class'] = pred
            df['pred_timestamp'] = str(datetime.datetime.now())
            df.to_csv('results.csv', mode='a', header=False)

        with open('data_copy.csv', 'a', newline="") as writepred:
            write_pred = csv.writer(writepred)
            for i in new_rows:
                write_pred.writerow(i)

    time.sleep(interval)

periodic_check(0.1)